In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

1.15.0


## constant & variable / operation / session

In [3]:
#상수 정의
a = tf.constant(1234)
b = tf.constant(5000)

#계산 정의
add_op = a+b

#세션 시작
sess = tf.Session()
res = sess.run(add_op)
print(res)

6234


In [8]:
#상수 정의
a = tf.constant(2)
b = tf.constant(3)
c = tf.constant(4)

#계산 정의
calc1_op = a + b * c
calc2_op = (a + b) * c

#세션 시작
sess = tf.Session()
res1 = sess.run(calc1_op)
print(res1)
res2 = sess.run(calc2_op)
print(res2)

14
20


In [27]:
#상수 정의
a = tf.constant(100)
b = tf.constant(50)

#계산 정의
add_op = a + b

#변수 정의
v = tf.Variable(0)

#변수 v에 add_op의 결과 대입하기
let_op = tf.assign(v, add_op)

#세션 시작
sess = tf.Session()

#변수 초기화(메모리에 값 생성)
sess.run(tf.global_variables_initializer())

#계산식 실행
sess.run(let_op)
print(sess.run(v))

150


## placeholder

In [28]:
#placeholder 정의(정수 자료형 3개를 가진 배열)
a = tf.placeholder(tf.int32, [3]) 
b = tf.constant(2)
x2_op = a* b

#세션 시작 
sess = tf.Session() 
# placeholder에 값 넣고 실행하기
r1 = sess.run(x2_op, feed_dict = {a:[1, 2, 3]})
print(r1)
r2 = sess.run(x2_op, feed_dict = {a:[10, 20, 10]})
print(r2)

[2 4 6]
[20 40 20]


- tensoflow.sqrt(x): x의 제곱근을 계산
- tensoflow.reduce_mean(x): x의 평균을 계산
- tensoflow.square(x): x의 제곱을 계산
- random_uniform([1], 0, 10,…) : 0에서 10 사이에서 임의의 수 1개 생성 반환
- Variable() : 변수의 값을 지정 

In [29]:
tf.random_uniform([1], 0, 10)

<tf.Tensor 'random_uniform:0' shape=(1,) dtype=float32>

In [30]:
#placeholder 데이터 크기가 미정이라면 [None] 입력
a = tf.placeholder(tf.int32, [None])
b = tf.constant(2)
x2_op = a*b

sess = tf.Session()
r1 = sess.run(x2_op, feed_dict = {a:[1, 2, 3, 4, 5]})
print(r1)
r2 = sess.run(x2_op, feed_dict = {a:[20, 30]})
print(r2)

[ 2  4  6  8 10]
[40 60]


## Tensor Board

In [32]:
# 데이터 플로우 그래프 구축하기  
a = tf.constant(20, name="a")
b = tf.constant(30, name="b")
mul_op = a * b
# 세션 생성하기  
sess = tf.Session()
# TensorBoard 사용하기  
tw = tf.summary.FileWriter("log_dir", graph=sess.graph)
# 세션 실행하기 
print(sess.run(mul_op))

600


# 최소제곱법

In [33]:
import numpy as np

x = [2, 4, 6, 8]
y = [81, 93, 91, 97]

mx = np.mean(x)
my = np.mean(y)

print("x의 평균값:", mx)
print("y의 평균값:", my)

x의 평균값: 5.0
y의 평균값: 90.5


In [35]:
divisor = sum([(mx-i)**2 for i in x])
def top(x, mx, y, my):
    d = 0
    for i in range(len(x)):
        d += (x[i] -mx) * (y[i] - my)
    return d

In [38]:
a = top(x, mx, y, my) / divisor
b = my - mx*a

In [39]:
a

2.3

In [40]:
b

79.0

## 평균 제곱근 오차

In [45]:
#임의의 기울기와 절편
ab = [3, 76]

data = list(zip(x, y))
data

[(2, 81), (4, 93), (6, 91), (8, 97)]

In [46]:
x = [i[0] for i in data]
y = [i[1] for i in data]

In [47]:
#임의의 기울기와 절편의 모델로부터 예측값을 반환하는 함수
def predict(x):
    return ab[0] * x + ab[1]

In [54]:
#평균 제곱근 오차 반환 함수
def rmse(p, a):
    return np.sqrt(((p - a) ** 2).mean())

In [55]:
def rmse_val(predict_result, y):
    return rmse(np.array(predict_result), np.array(y))

In [56]:
predict_result = []

for i in range(len(x)):
    predict_result.append(predict(x[i]))
    print("공부한 시간 = %.f, 실제 점수 = %.f, 예측점수 = %.f" % (x[i], y[i], predict(x[i])))
    
print("rmse 최종값: " + str(rmse_val(predict_result, y)))

공부한 시간 = 2, 실제 점수 = 81, 예측점수 = 82
공부한 시간 = 4, 실제 점수 = 93, 예측점수 = 88
공부한 시간 = 6, 실제 점수 = 91, 예측점수 = 94
공부한 시간 = 8, 실제 점수 = 97, 예측점수 = 100
rmse 최종값: 3.3166247903554


# 경사하강법

경사하강법(gradient decent) 코드 구현  
경사 하강법은 오차의 변화에 따라 이차 함수 그래프를 만들고 적절한 학습률을 설정해 미분 값이 0인 지점을 구하는 것입니다.
y 절편 b의 값도  b 값이 너무 크면 오차도 함께 커지고 너무 작아도 오차가 커집니다. 
최적의 b 값을 구할 때 역시 경사 하강법을 사용합니다.
- GradientDescentOptimizer() : 경사 하강법 함수
텐서플로는 session 함수를 이용해 구동에 필요한 리소스를 컴퓨터에 할당하고 이를 실행시킬 준비를 합니다. 
Session을 통해 구현될 함수를 텐서플로에서는 ‘그래프’라고 부른다
Session이 할당되면 session.run('그래프명')의 형식으로 해당 함수를 실행시킵니다. 
- global_variables_initializer() :  변수를 초기화하는 함수 
- gradient_decent :  총 필요한 수만큼 반복하여 실행 

In [57]:
data

[(2, 81), (4, 93), (6, 91), (8, 97)]

In [58]:
x_data = [i[0] for i in data]
y_data = [i[1] for i in data]

In [59]:
x_data

[2, 4, 6, 8]

In [60]:
y_data

[81, 93, 91, 97]

In [65]:
#임의의 기울기와 y절편의 값으로 변수 정의
a = tf.Variable(tf.random_uniform([1], 0, 10, dtype = tf.float64, seed = 0))  #0과 10 사이의 난수 1개 생성
b = tf.Variable(tf.random_uniform([1], 0, 100, dtype = tf.float64, seed = 0))

y = a*x_data + b # 1차방정식의 계산식 정의

#오차 계산(평균 제곱근 오차 공식)
rmse = tf.sqrt(tf.reduce_mean(tf.square(y-y_data)))

#학습률
learning_rate = 0.1

#오차 rmse 값이 최소인 값 찾는 식 정의
gradient_descent = tf.train.GradientDescentOptimizer(learning_rate).minimize(rmse)   #경사하강법!

#텐서플로우로 학습
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())      #변수를 메모리에 생성
    for step in range(2001):
        sess.run(gradient_descent)
        if step % 100 == 0:
            print("Epoch: %.f, RMSE = %.4f, 기울기 a=%.4f, 절편 b=%.4f" % (step, sess.run(rmse), sess.run(a), sess.run(b)))

Epoch: 0, RMSE = 30.2139, 기울기 a=7.5235, 절편 b=80.5984
Epoch: 100, RMSE = 2.8860, 기울기 a=2.2299, 절편 b=79.4181
Epoch: 200, RMSE = 2.8826, 기울기 a=2.2601, 절편 b=79.2379
Epoch: 300, RMSE = 2.8815, 기울기 a=2.2773, 절편 b=79.1353
Epoch: 400, RMSE = 2.8811, 기울기 a=2.2871, 절편 b=79.0770
Epoch: 500, RMSE = 2.8810, 기울기 a=2.2927, 절편 b=79.0438
Epoch: 600, RMSE = 2.8810, 기울기 a=2.2958, 절편 b=79.0249
Epoch: 700, RMSE = 2.8810, 기울기 a=2.2976, 절편 b=79.0142
Epoch: 800, RMSE = 2.8810, 기울기 a=2.2987, 절편 b=79.0081
Epoch: 900, RMSE = 2.8810, 기울기 a=2.2992, 절편 b=79.0046
Epoch: 1000, RMSE = 2.8810, 기울기 a=2.2996, 절편 b=79.0026
Epoch: 1100, RMSE = 2.8810, 기울기 a=2.2998, 절편 b=79.0015
Epoch: 1200, RMSE = 2.8810, 기울기 a=2.2999, 절편 b=79.0008
Epoch: 1300, RMSE = 2.8810, 기울기 a=2.2999, 절편 b=79.0005
Epoch: 1400, RMSE = 2.8810, 기울기 a=2.3000, 절편 b=79.0003
Epoch: 1500, RMSE = 2.8810, 기울기 a=2.3000, 절편 b=79.0002
Epoch: 1600, RMSE = 2.8810, 기울기 a=2.3000, 절편 b=79.0001
Epoch: 1700, RMSE = 2.8810, 기울기 a=2.3000, 절편 b=79.0001
Epoch: 1800, RMSE = 2

# 다중 선형 회귀

In [66]:
x1_data = [2, 4, 6, 8]
x2_data = [0, 4, 2, 3]
y_data = [81, 93, 91, 97]

#임의의 기울기와 y절편의 값으로 변수 정의
a1 = tf.Variable(tf.random_uniform([1], 0, 10, dtype = tf.float64, seed = 0))
a2 = tf.Variable(tf.random_uniform([1], 0, 10, dtype = tf.float64, seed = 0))#0과 10 사이의 난수 1개 생성
b = tf.Variable(tf.random_uniform([1], 0, 100, dtype = tf.float64, seed = 0))

y = a1*x1_data + a2*x2_data + b # 1차방정식의 계산식 정의

#오차 계산(평균 제곱근 오차 공식)
rmse = tf.sqrt(tf.reduce_mean(tf.square(y-y_data)))

#학습률
learning_rate = 0.1

#오차 rmse 값이 최소인 값 찾는 식 정의
gradient_descent = tf.train.GradientDescentOptimizer(learning_rate).minimize(rmse)   #경사하강법!

#텐서플로우로 학습
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())      #변수를 메모리에 생성
    for step in range(2001):
        sess.run(gradient_descent)
        if step % 100 == 0:
            print("Epoch: %.f, RMSE = %.4f, 기울기 a1=%.4f, 기울기 a2=%.4f, 절편 b=%.4f" % (step, sess.run(rmse), sess.run(a1), sess.run(a2), sess.run(b)))

Epoch: 0, RMSE = 49.1842, 기울기 a1=7.5270, 기울기 a2=7.8160, 절편 b=80.5980
Epoch: 100, RMSE = 1.8368, 기울기 a1=1.1306, 기울기 a2=2.1316, 절편 b=78.5119
Epoch: 200, RMSE = 1.8370, 기울기 a1=1.1879, 기울기 a2=2.1487, 절편 b=78.1057
Epoch: 300, RMSE = 1.8370, 기울기 a1=1.2122, 기울기 a2=2.1571, 절편 b=77.9352
Epoch: 400, RMSE = 1.8370, 기울기 a1=1.2226, 기울기 a2=2.1607, 절편 b=77.8636
Epoch: 500, RMSE = 1.8370, 기울기 a1=1.2269, 기울기 a2=2.1622, 절편 b=77.8335
Epoch: 600, RMSE = 1.8370, 기울기 a1=1.2288, 기울기 a2=2.1628, 절편 b=77.8208
Epoch: 700, RMSE = 1.8370, 기울기 a1=1.2295, 기울기 a2=2.1631, 절편 b=77.8155
Epoch: 800, RMSE = 1.8370, 기울기 a1=1.2299, 기울기 a2=2.1632, 절편 b=77.8133
Epoch: 900, RMSE = 1.8370, 기울기 a1=1.2300, 기울기 a2=2.1632, 절편 b=77.8124
Epoch: 1000, RMSE = 1.8370, 기울기 a1=1.2301, 기울기 a2=2.1633, 절편 b=77.8120
Epoch: 1100, RMSE = 1.8370, 기울기 a1=1.2301, 기울기 a2=2.1633, 절편 b=77.8118
Epoch: 1200, RMSE = 1.8370, 기울기 a1=1.2301, 기울기 a2=2.1633, 절편 b=77.8117
Epoch: 1300, RMSE = 1.8370, 기울기 a1=1.2301, 기울기 a2=2.1633, 절편 b=77.8117
Epoch: 1400, RMSE